## 7.19 结构与类

如今，编程教科书推荐使用面向对象编程，使软件开发更加清晰和模块化。
对象是结构和类的实例。
面向对象的编程风格对程序性能，既有正面又有负面的影响。积极影响是：

- 如果一组变量是相同结构或类的成员，则它们一起使用，也存储在一起。这使数据缓存更有效。
- 类成员变量不需要作为参数传递给类成员函数，避免了参数传输的开销。

面向对象编程的负面影响是：

- 非静态成员函数有一个'`this`'指针，它作为隐式参数传递给函数。所有非静态成员函数都会产生'`this`'参数传输的开销。
- '`this`'指针占用一个寄存器。寄存器是32位系统中的稀缺资源。
- 虚拟成员函数效率较低（请参阅第55页）。

**关于面向对象编程的正面影响，还是负面影响占主导地位，没有一般性的定论。**
至少，可以说使用类和成员函数，代价并不昂贵。
你可以使用面向对象的编程风格：
- 如果它对程序的逻辑结构和清晰度有好处
- 只要在程序的最关键部分避免过多的函数调用，

单纯结构的使用（没有成员函数）对性能没有负面影响。

## 7.20 类数据成员（实例变量）

The data members of a class or structure are stored consecutively in the order in which they
are declared whenever an instance of the class or structure is created. There is no
performance penalty for organizing data into classes or structures. Accessing a data
member of a class or structure object takes no more time than accessing a simple variable.

Most compilers will align data members to round addresses in order to optimize access, as
given in the following table.

Type | size, bytes  | alignment, bytes
 ---  | ---  | ---  |
bool |  1 |  1
char, signed or unsigned  | 1  | 1
short int, signed or unsigned  | 2  | 2
int, signed or unsigned  | 4  | 4
64-bit integer, signed or unsigned  | 8  | 8
pointer or reference, 32-bit mode  | 4  | 4
pointer or reference, 64-bit mode  | 8  | 8
float  | 4  | 4
double  | 8 |  8
long double  | 8, 10, 12 or 16 |  8 or 16

Table 7.2. Alignment of data members.

This alignment can cause holes of unused bytes in a structure or class with members of
mixed sizes. For example:

```cpp
// Example 7.39a
struct S1 {
   short int a; // 2 bytes. first byte at 0, last byte at 1
   // 6 unused bytes
   double b; // 8 bytes. first byte at 8, last byte at 15
   int d; // 4 bytes. first byte at 16, last byte at 19
   // 4 unused bytes
};
S1 ArrayOfStructures[100];
```
This reordering has made the structure 8 bytes smaller and the array 800 bytes smaller.

Structure and class objects can often be made smaller by reordering the data members. If
the class has at least one virtual member functions then there is a pointer to a virtual table
before the first data member or after the last member. This pointer is 4 bytes in 32-bit
systems and 8 bytes in 64-bit systems. If you are in doubt how big a structure or each of its 
members are then you may make some tests with the `sizeof` operator. The value returned
by the `sizeof` operator includes any unused bytes in the end of the object.

The code for accessing a data member is more compact if the offset of the member relative
to the beginning of the structure or class is less than 128 because the offset can be
expressed as an 8-bit signed number. If the offset relative to the beginning of the structure
or class is 128 bytes or more then the offset has to be expressed as a 32-bit number (the
instruction set has nothing between 8 bit and 32 bit offsets). Example:

```cpp
// Example 7.40
class S2 {
   public:
   int a[100]; // 400 bytes. first byte at 0, last byte at 399
   int b; // 4 bytes. first byte at 400, last byte at 403
   int ReadB() {return b;}
};
```

The offset of `b` is 400 here. Any code that accesses b through a pointer or a member
function such as `ReadB` needs to code the offset as a 32-bit number. If a and b are
swapped then both can be accessed with an offset that is coded as an 8-bit signed number,
or no offset at all. This makes the code more compact so that the code cache is used more
efficiently. It is therefore recommended that big arrays and other big objects come last in a
structure or class declaration and the most often used data members come first. If it is not
possible to contain all data members within the first 128 bytes then put the most often used
members in the first 128 bytes.


## 7.21 Class member functions (methods)

Each time a new object of a class is declared or created it will generate a new instance of
the data members. But each member function has only one instance. The function code is
not copied because the same code can be applied to all instances of the class.

Calling a member function is as fast as calling a simple function with a pointer or reference
to a structure. For example:

```cpp
// Example 7.41
class S3 {
   public:
   int a;
   int b;
   int Sum1() {return a + b;}
};
int Sum2(S3 * p) {return p->a + p->b;}
int Sum3(S3 & r) {return r.a + r.b;}
```

The three functions `Sum1`, `Sum2` and `Sum3` are doing exactly the same thing and they are
equally efficient. If you look at the code generated by the compiler, you will notice that some
compilers will make exactly identical code for the three functions. `Sum1` has an implicit
`'this'` pointer which does the same thing as `p` and `r` in `Sum2` and `Sum3`. Whether you want
to make the function a member of the class or give it a pointer or reference to the class or
structure is simply a matter of programming style. Some compilers make `Sum1` slightly more
efficient than `Sum2` and `Sum3` in 32-bit Windows by transferring `'this'` in a register rather
than on the stack.


## 7.22 Virtual member functions

Virtual functions are used for implementing polymorphic classes. Each instance of a
polymorphic class has a pointer to a table of pointers to the different versions of the virtual
functions. This so-called virtual table is used for finding the right version of the virtual
function at runtime. Polymorphism is one of the main reasons why object oriented programs
can be less efficient than non-object oriented programs. If you can avoid virtual functions
then you can obtain most of the advantages of object oriented programming without paying
the performance costs.

The time it takes to call a virtual member function is a few clock cycles more than it takes to
call a non-virtual member function, provided that the function call statement always calls the
same version of the virtual function. If the version changes then you may get a misprediction
penalty of 10 - 20 clock cycles. The rules for prediction and misprediction of virtual function
calls is the same as for switch statements, as explained on page 44.

The dispatching mechanism can be bypassed when the virtual function is called on an
object of known type, but you cannot always rely on the compiler bypassing the dispatch
mechanism even when it would be obvious to do so. See page 75.

Runtime polymorphism is needed only if it cannot be known at compile time which version
of a polymorphic member function is called. If virtual functions are used in a critical part of a
program then you may consider whether it is possible to obtain the desired functionality
without polymorphism or with compile-time polymorphism.

It is sometimes possible to obtain the desired polymorphism effect with templates instead of
virtual functions. The template parameter should be a class containing the functions that
have multiple versions. This method is faster because the template parameter is always
resolved at compile time rather than at runtime. Example 7.47 on page 59 shows an
example of how to do this. Unfortunately, the syntax is so kludgy that it may not be worth
the effort.


## 7.23 Runtime type identification (RTTI)

Runtime type identification adds extra information to all class objects and is not efficient. If
the compiler has an option for RTTI then turn it off and use alternative implementations.

## 7.24 Inheritance

An object of a derived class is implemented in the same way as an object of a simple class
containing the members of both parent and child class. Members of parent and child class
are accessed equally fast. In general, you can assume that there is hardly any performance
penalty to using inheritance.

There may be a slight degradation in code caching for the following reasons:

- The size of the parent class data members is added to the offset of the child class
members. The code that accesses data members with a total offset bigger than 127
bytes is slightly less compact. See page 54.

- The member functions of parent and child are typically stored in different modules.
This may cause a lot of jumping around and less efficient code caching. This
problem can be solved by making sure that functions which are called near each
other are also stored near each other. See page 90 for details.

Inheritance from multiple parent classes in the same generation can cause complications
with member pointers and virtual functions or when accessing an object of a derived class
through a pointer to one of the base classes. You may avoid multiple inheritance by making
objects inside the derived class:
```cpp
// Example 7.42a. Multiple inheritance
class B1; class B2;
class D : public B1, public B2 {
public:
   int c;
};
```
Replace with:
```cpp
// Example 7.42b. Alternative to multiple inheritance
class B1; class B2;
class D : public B1 {
public:
   B2 b2;
   int c;
};
```

## 7.25 Constructors and destructors

A constructor is implemented internally as a member function which returns a reference to
the object. The allocation of memory for a new object is not necessarily done by the
constructor itself. Constructors are therefore as efficient as any other member functions.
This applies to default constructors, copy constructors, and any other constructors.

A class doesn't need a constructor. A default constructor is not needed if the object doesn't
need initialization. A copy constructor is not needed if the object can be copied simply by
copying all data members. A simple constructor may be inlined for improved performance.

A copy constructor may be called whenever an object is copied by assignment, as a
function parameter, or as a function return value. The copy constructor can be a time
consumer if it involves allocation of memory or other resources. There are various ways to
avoid this wasteful copying of memory blocks, for example:

- Use a reference or pointer to the object instead of copying it
- Use a "move constructor" to transfer ownership of the memory block. This requires a
compiler with C++0x support.
- Make a member function or friend function or operator that transfers ownership of
the memory block from one object to another. The object that looses ownership of
the memory block should have its pointer set to NULL. There should of course be a
destructor that destroys any memory block that the object owns.

A destructor is as efficient as a member function. Do not make a destructor if it is not
necessary. A virtual destructor is as efficient as a virtual member function. See page 55.